# Final Project PCV: Deteksi Objek dengan YOLOv5 dan Custom Dataset

Mendeteksi objek dengan YOLOv5 dengan menggunakan model dari custom dataset yang sudah dibuat pada Roboflow

### Periksa instalasi YOLOv5

In [1]:
import torch
import utils
display = utils.notebook_init()

YOLOv5  v6.1-267-g264aafa Python-3.10.4 torch-1.11.0+cu113 CPU


Setup complete  (4 CPUs, 7.0 GB RAM, 277.8/290.8 GB disk)


### Tes deteksi menggunakan webcam dan model yolov5s (metode 1)

In [ ]:
!python detect.py --weights yolov5s.pt --source 1
# --source: sumber yang digunakan untuk deteksi
# --weights: model yang digunakan untuk deteksi

### Tes deteksi menggunakan webcam dan model yolov5s (metode 2)

In [ ]:
import cv2 
import torch
import numpy as np 

model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
model.img = 360
cap = cv2.VideoCapture(1)
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    result = model(frame)
    DetectionBoxes = result.pandas().xyxy[0].values.tolist()

    for Box in DetectionBoxes:
      # Bounding Box
      xmin,ymin,xmax,ymax,conf,cl,name = Box
      pmin =(int(xmin),int(ymin))
      pmax =(int(xmax),int(ymax))
      frame = cv.rectangle(frame, pmin, pmax,(255,255,0),3)
      pPos = (int(xmin),int(ymin))
      # Font
      fontScale = 2
      color = (255, 255, 255)
      thickness = 2
      font = cv.FONT_HERSHEY_DUPLEX
      frame = cv.putText(frame,name, pPos, font,fontScale, color,thickness, cv.LINE_AA)
    cv2.imshow('Frame',frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
  else: 
    break
cap.release()
cv2.destroyAllWindows()

### Training custom dataset

In [ ]:
!python train.py --batch 25 --epochs 100 --data data.yaml --cfg custom_data.yaml --weights '' --name train_result --cache
# --epochs: jumlah pengulangan training
# --data: dataset yang digunakan untuk training
# --cfg: file konfigurasi yang digunakan untuk training
# --weights: model yang digunakan sebagai patokan untuk training, '' untuk training dataset dari awal
# --name: nama folder hasil training disimpan
# --cache: caching ke RAM untuk mempercepat proses training

### Tes deteksi menggunakan model dari dataset yang sudah di-train (metode 1)

In [ ]:
!python detect.py --img 480 --weights best_result14.pt --source 1

### Tes deteksi menggunakan model dari dataset yang sudah di-train (metode 2)

In [ ]:
import cv2 as cv
import torch
import numpy as np 

# Model yang digunakan untuk deteksi
path = "yolov5s.pt"
model = torch.hub.load('', 'custom', path=path, source='local')
# Source yang digunakan untuk deteksi
cap = cv.VideoCapture(1)
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

# Baca source sampai selesai
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    result = model(frame) # Inference
    DetectionBoxes = result.pandas().xyxy[0].values.tolist()

    for Box in DetectionBoxes:
      # Bounding Box
      xmin,ymin,xmax,ymax,conf,cl,name = Box
      pmin =(int(xmin),int(ymin))
      pmax =(int(xmax),int(ymax))
      frame = cv.rectangle(frame, pmin, pmax,(255,255,0),3)
      pPos = (int(xmin),int(ymin))
      # Font
      fontScale = 2
      color = (255, 255, 255)
      thickness = 2
      font = cv.FONT_HERSHEY_DUPLEX
      frame = cv.putText(frame,name, pPos, font,fontScale, color,thickness, cv.LINE_AA)
    # Menampilkan window capture
    cv.imshow('Frame',frame)
    # Tombol Q untuk keluar dari window capture
    if cv.waitKey(25) & 0xFF == ord('q'):
      break
  # Keluar dari loop
  else: 
    break
cap.release()
# Hapus Semua frame
cv.destroyAllWindows()